# Versiunea CPU

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Master/AI
!ls

# Import Libraries

In [ ]:
!pip install transformers

In [ ]:
import time
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import gensim.downloader as api
import itertools
from transformers import BertTokenizer, TFBertModel
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Choose dataset, embedding method and algorithm

In [ ]:
#@title Dataset

dataset_name = 'Hateval2019' #@param ["Hateval2019", "Davidson"] {allow-input: true}

print(dataset_name)

In [ ]:
#@title Embedding Method

embedding_method_name = 'ELMo' #@param ["ELMo", "FastText", "Word2Vec", "GloVe", "BERT"] {allow-input: true}

print(embedding_method_name)

# Load Datasets

In [ ]:
def load_hateval2019_dataset(file_path: str):
    """
    Loads the Hateval2019 dataset from a CSV file located at the specified file path
    and returns a Pandas DataFrame containing the loaded data.

    Args:
        file_path (str): The path to the CSV file containing the Hateval2019 dataset.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the loaded data.
    """
    column_names = ["id", "text", "HS", "TR", "AG"]
    df = pd.read_csv(file_path, names=column_names, header=0)
    return df

In [ ]:
# def load_davidson_dataset(file_path: str):
#     """
#     Loads the Davidson dataset from a CSV file located at the specified file path
#     and returns a Pandas DataFrame containing the loaded data.

#     Args:
#         file_path (str): The path to the CSV file containing the Davidson dataset.

#     Returns:
#         pd.DataFrame: A Pandas DataFrame containing the loaded data.
#     """
#     column_names = ["count", "hate_speech", "offensive_language", "neither", "class", "tweet"]
#     df = pd.read_csv(file_path, names=column_names, header=0)
#     return df

In [ ]:
# Load Hateval2019 dataset and print execution time
start = time.time()
hateval2019_dataset = load_hateval2019_dataset("/content/drive/MyDrive/Master/AI/DATASETS/HATEVAL/hateval2019_en.csv")
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
# # Load Davidson dataset and print execution time
# start = time.time()
# davidson_dataset = load_davidson_dataset("/content/drive/MyDrive/Master/AI/DATASETS/TWITTER/labeled_data.csv")
# end = time.time()
# print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
if dataset_name == 'Hateval2019':
  data = hateval2019_dataset['text']
  labels = hateval2019_dataset['HS']
# elif dataset_name == 'Davidson':
#   data = davidson_dataset['tweet']
#   labels = davidson_dataset['hate_speech']

# Preprocess Data

In [ ]:
def preprocess_text(text):
    """
    Preprocesses text data by performing the following steps:
    - Removes HTML tags and URLs
    - Tokenizes the text
    - Converts tokens to lowercase
    - Removes stopwords
    - Lemmatizes tokens

    Args:
        text (str): The text data to preprocess

    Returns:
        str: A preprocessed string
    """
    # Remove HTML tags and URLs
    text = re.sub('<[^<]+?>', '', text)
    text = re.sub(r'http\S+', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the preprocessed tokens back into a string
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

In [ ]:
data = data.apply(preprocess_text)

# Load Embeddings

In [ ]:
def load_glove_model(file_path):
    """
    Loads the GloVe model from a file located at the specified file path and 
    returns a dictionary containing the word vectors from the model.

    Args:
        file_path (str): The path to the file containing the GloVe model.

    Returns:
        dict: A Python dictionary object containing the word vectors from the 
        GloVe model. Each key in the dictionary represents a word, and the value 
        associated with the key is a numpy array that contains the corresponding 
        word vector.
    """
    glove_model = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

In [ ]:
def elmo_vectors(text, batch_size=8):
    """
    Computes ELMo embeddings for the given text using a pre-trained ELMo model.

    Args:
        text (numpy.ndarray): A 1-dimensional numpy array containing the text for 
        which ELMo embeddings are to be computed.
        batch_size (int, optional): The batch size to use when computing embeddings. 
        Defaults to 8.

    Returns:
        numpy.ndarray: A 2-dimensional numpy array containing the ELMo embeddings 
        for the given text. Each row in the array represents a sentence, and each 
        column represents a dimension in the embedding space.
    """
    num_batches = len(text) // batch_size + (1 if len(text) % batch_size > 0 else 0)
    embeddings = []
    
    for i in range(num_batches):
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, len(text))
        batch_text = text[start_index:end_index]
        batch_embeddings = elmo(tf.convert_to_tensor(batch_text.tolist()), training=False)
        embeddings.append(batch_embeddings.numpy())

    return np.concatenate(embeddings, axis=0)

In [ ]:
def fasttext_word2vec_vectors(text, model):
    """
    Computes FastText word embeddings for the given text using a pre-trained 
    FastText model.

    Args:
        text (list): A list of strings containing the text for which FastText 
        embeddings are to be computed.
        model (fasttext.FastText._FastText): The pre-trained FastText model to 
        use for computing embeddings.

    Returns:
        numpy.ndarray: A 2-dimensional numpy array containing the FastText 
        embeddings for the given text. Each row in the array represents a 
        sentence, and each column represents a dimension in the embedding space.
    """
    embeddings = []
    zero_vector = np.zeros(model.vector_size)

    for sentence in text:
        words = sentence.split()
        sentence_embeddings = [model[word] for word in words if word in model]
        
        if not sentence_embeddings:
            sentence_embeddings = [zero_vector]
        
        embeddings.append(np.mean(sentence_embeddings, axis=0))

    return np.array(embeddings)


In [ ]:
def glove_vectors(text):
    """
    Computes GloVe embeddings for the given text using a pre-loaded GloVe model.

    Args:
        text (list): A list of strings containing the text for which GloVe 
        embeddings are to be computed.

    Returns:
        numpy.ndarray: A 2-dimensional numpy array containing the GloVe embeddings 
        for the given text. Each row in the array represents a sentence, and each 
        column represents a dimension in the embedding space.
    """
    embeddings = []
    
    # Get the vector size from the first word in the dictionary
    vector_size = len(next(iter(glove_model.values())))
    zero_vector = np.zeros(vector_size)

    for sentence in text:
        words = sentence.split()
        sentence_embeddings = [glove_model[word] for word in words if word in glove_model]

        if not sentence_embeddings:
            sentence_embeddings = [zero_vector]

        embeddings.append(np.mean(sentence_embeddings, axis=0))

    return np.array(embeddings)


In [ ]:
def bert_vectors(text, batch_size=32):
    """
    Computes BERT embeddings for the given text using a pre-trained BERT model.

    Args:
        text (list): A list of strings containing the text for which BERT embeddings 
        are to be computed.
        batch_size (int, optional): The batch size to use when computing embeddings. 
        Defaults to 32.

    Returns:
        numpy.ndarray: A 2-dimensional numpy array containing the BERT embeddings 
        for the given text. Each row in the array represents a sentence, and each 
        column represents a dimension in the embedding space.
    """
    # Load the BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = TFBertModel.from_pretrained("bert-base-uncased")

    # Compute the number of batches
    num_batches = len(text) // batch_size + (1 if len(text) % batch_size > 0 else 0)
    embeddings = []

    for i in range(num_batches):
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, len(text))
        batch_text = text[start_index:end_index]

        # Tokenize the batch of text and generate input tensors for the model
        inputs = tokenizer(batch_text.tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)

        # Generate embeddings for the batch using the BERT model
        outputs = model(inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings.append(batch_embeddings.numpy())

    return np.concatenate(embeddings, axis=0)

In [ ]:
if embedding_method_name == "ELMo":
  elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False)
  embeddings_func = elmo_vectors
elif embedding_method_name == "FastText":
  fasttext_model = api.load("fasttext-wiki-news-subwords-300")
  embeddings_func = lambda text: fasttext_word2vec_vectors(text, fasttext_model)
elif embedding_method_name == "Word2Vec":
  word2vec_model = api.load("word2vec-google-news-300")
  embeddings_func = lambda text: fasttext_word2vec_vectors(text, word2vec_model)
elif embedding_method_name == "GloVe":
  glove_model = load_glove_model('/content/drive/MyDrive/Master/AI/glove.6B.50d.txt')
  embeddings_func = glove_vectors
elif embedding_method_name == "BERT":
  bert = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)
  embeddings_func = bert_vectors

In [ ]:
embeddings = embeddings_func(data)
print(f"{embedding_method_name} embeddings shape:", embeddings.shape)

# Split Data

In [ ]:
X = embeddings
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Machine Learning Algorithms

## Gaussian Naive Bayes

In [ ]:
start = time.time()
gnb = GaussianNB()
gnb.fit(X_train, y_train)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
start = time.time()
gnb_accuracy = gnb.score(X_test, y_test)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

## Bernoulli Naive Bayes

In [ ]:
start = time.time()
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
start = time.time()
bnb_accuracy = bnb.score(X_test, y_test)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

## Logistic Regression

In [ ]:
start = time.time()
lr = LogisticRegression()
lr.fit(X_train, y_train)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
start = time.time()
lr_accuracy = lr.score(X_test, y_test)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

## Support Vector Machines

In [ ]:
start = time.time()
svm = SVC()
svm.fit(X_train, y_train)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
start = time.time()
svm_accuracy = svm.score(X_test, y_test)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

## Random Forest

In [ ]:
start = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf.fit(X_train, y_train)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

In [ ]:
start = time.time()
rf_accuracy = rf.score(X_test, y_test)
end = time.time()
print(f"Execution time: {end-start:.2f} seconds")

### Accuracies

In [ ]:
print(f"Gaussian Naive Bayes Train Accuracy: {gnb.score(X_train, y_train) * 100:.2f}")
print(f"Bernoulli Naive Bayes Train Accuracy: {bnb.score(X_train, y_train) * 100:.2f}")
print(f"Logistic Regression Train Accuracy: {lr.score(X_train, y_train) * 100:.2f}")
print(f"SVM Train Accuracy: {svm.score(X_train, y_train) * 100:.2f}")
print(f"Random Train Forest Accuracy: {rf.score(X_train, y_train) * 100:.2f}")

In [ ]:
print(f"Gaussian Naive Bayes Test Accuracy: {gnb_accuracy * 100:.2f}")
print(f"Bernoulli Naive Bayes Test Accuracy: {bnb_accuracy * 100:.2f}")
print(f"Logistic Regression Test Accuracy: {lr_accuracy * 100:.2f}")
print(f"SVM Test Accuracy: {svm_accuracy * 100:.2f}")
print(f"Random Test Forest Accuracy: {rf_accuracy * 100:.2f}")